# Hubbard12

## Introduction

### Init

In [23]:
import classifim.bench
import classifim.bench.fidelity
import classifim.bench.metric
import classifim.bench.plot_tools
import classifim.utils
import classifim_gen.fil24_hamiltonian
import classifim_gen.gs_cache
import classifim_gen.gs_utils
import classifim_gen.hubbard_hamiltonian
import classifim_gen.io
import concurrent.futures
import functools
import importlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy.sparse.linalg
import sys
from tqdm import tqdm

The following settings were used to generate the images for the main paper. They are commented out (the cell is made "raw" using CTRL+R, use CTRL+Y to make it "code") since they require a LaTeX installation with some fonts and packages used by matplotlib.

In [15]:
SM_NAME = "Hubbard12"
DATA_DIR0 = classifim.utils.find_data_dir()
assert os.path.isdir(DATA_DIR0)
DATA_DIR = classifim.utils.maybe_create_subdir(DATA_DIR0, SM_NAME.lower())

HF_DATA_DIR0 = classifim.utils.find_data_dir("hf_data_dir")
HF_DATA_DIR = classifim.utils.maybe_create_subdir(HF_DATA_DIR0, "hubbard_12")

In [6]:
ham_family = classifim_gen.hubbard_hamiltonian.Hubbard1DFamily(dtype_float=np.float64, xp=np)
param_conversions = classifim_gen.hubbard_hamiltonian.HubbardParamsConversions(ham_family)

In [7]:
@functools.cache
def get_lanczos_cache():
    """
    Returns lanczos_cache without a way to compute new values.
    """
    DATA_LANCZOS_DIR = os.path.join(DATA_DIR, "lanczos")
    assert os.path.exists(DATA_LANCZOS_DIR)
    return classifim_gen.gs_cache.GroundStateCache(
        compute_ground_state=None,
        param_keys=ham_family.PARAM_NAMES,
        ham_name=SM_NAME.lower(),
        save_path=DATA_LANCZOS_DIR,
        load_meta=True,
        filename_source=classifim_gen.gs_cache.FS_FILESYSTEM)

### About Hubbard12

**Hubbard Hamiltonian**

Consider a lattice $\Lambda = \mathbb{Z}^2 / a_{\Lambda} \mathbb{Z}^2$ where
$a_{\Lambda}$ is an invertible $2\times 2$ matrix with integer coefficients.
The number of sites is $\left|\Lambda\right| = \left|\det(a_{\Lambda})\right|$.
Each site $j \in \Lambda$ has neighbors
$$\text{NN}(j) = \left\{j + v: v \in \left\{
\left(\begin{smallmatrix}-1\\0\end{smallmatrix}\right),
\left(\begin{smallmatrix}0\\-1\end{smallmatrix}\right),
\left(\begin{smallmatrix}0\\1\end{smallmatrix}\right),
\left(\begin{smallmatrix}1\\0\end{smallmatrix}\right)
\right\}\right\}$$
and diagonal neighbours
$$\text{NNN}(j) = \left\{j + v: v \in \left\{
\left(\begin{smallmatrix}-1\\-1\end{smallmatrix}\right),
\left(\begin{smallmatrix}-1\\1\end{smallmatrix}\right),
\left(\begin{smallmatrix}1\\-1\end{smallmatrix}\right),
\left(\begin{smallmatrix}1\\1\end{smallmatrix}\right)
\right\}\right\}.$$
We introduce $\text{NN}(\Lambda)=\{(i,j): i\in \text{NN}(j)\}$, $\text{NNN}(\Lambda)=\{(i,j): i\in \text{NNN}(j)\}$
for respectively regular and diagonal edges of the lattice $\Lambda$.

Each site $j$ of the lattice $\Lambda$ has 2 slots for an electron enumerated by spin $\sigma\in\{\uparrow,\downarrow\}$
with creation and annihilation operators $c_{j\sigma}^\dagger$ and $c_{j\sigma}$. We consider the Hubbard Hamiltonian family given by
$$H = u H_{\text{int}} + t H_{\text{NN}} + t' H_{\text{NNN}},$$
where
$$
H_{\text{int}} = \sum_{i\in\Lambda} (n_{j\uparrow}-1/2) (n_{j\downarrow}-1/2),
\qquad n_{j\sigma} = c_{j\sigma}^{\dagger} c_{j\sigma},
$$
$$
H_{\text{NN}} = -\sum_{(i,j)\in\text{NN}(\Lambda),\sigma}c_{i\sigma}^{\dagger}c_{j\sigma},
\qquad H_{\text{NNN}} = -\sum_{(i,j)\in\text{NNN}(\Lambda),\sigma}c_{i\sigma}^{\dagger}c_{j\sigma}.
$$

Since $N_{\uparrow} = \sum_{j} n_{j\uparrow}$ and $N_{\downarrow} = \sum_{j} n_{j\downarrow}$
are symmetries of the Hamiltonian, we can restrict our attention to a sector with fixed
values of both. We pick $N_{\uparrow} = N_{\downarrow} = \left|\Lambda\right|/2$.

**Lattice choice**

We pick $\left|\det(a_{\Lambda})\right|=12$ since there are $924^2 = 853776$ configurations with $N_{\uparrow} = N_{\downarrow} = \left|\Lambda\right|/2$ at this lattice size (larger sizes will be too slow). There are the following keeping the NN graph bipartite:

* 10: $\left(\begin{smallmatrix}5&1\\5&-1\end{smallmatrix}\right),\left(\begin{smallmatrix}1&2\\5&0\end{smallmatrix}\right),\left(\begin{smallmatrix}1&3\\3&-1\end{smallmatrix}\right)$,
* 12: $\left(\begin{smallmatrix}6&1\\6&-1\end{smallmatrix}\right),\left(\begin{smallmatrix}3&2\\3&-2\end{smallmatrix}\right),\left(\begin{smallmatrix}4&2\\2&-2\end{smallmatrix}\right),\left(\begin{smallmatrix}1&4\\3&0\end{smallmatrix}\right),\left(\begin{smallmatrix}0&6\\2&0\end{smallmatrix}\right)$,
* 14: $\left(\begin{smallmatrix}7&1\\7&-1\end{smallmatrix}\right),\left(\begin{smallmatrix}1&2\\7&0\end{smallmatrix}\right),\left(\begin{smallmatrix}2&3\\4&-1\end{smallmatrix}\right)$,
* 16: $\left(\begin{smallmatrix}8&1\\8&-1\end{smallmatrix}\right),\left(\begin{smallmatrix}4&2\\4&-2\end{smallmatrix}\right),\left(\begin{smallmatrix}5&2\\3&-2\end{smallmatrix}\right),\left(\begin{smallmatrix}1&5\\3&-1\end{smallmatrix}\right),\left(\begin{smallmatrix}0&4\\4&0\end{smallmatrix}\right),\left(\begin{smallmatrix}0&8\\2&0\end{smallmatrix}\right),\left(\begin{smallmatrix}2&4\\4&0\end{smallmatrix}\right)$.



We pick $a_{\Lambda} = \left(\begin{smallmatrix}0&-3\\4&1\end{smallmatrix}\right)$, $\det(a_{\Lambda}) = 12$.
We number the vertices of the lattice $\Lambda$ by integers $0,\dots,11$, where integer $j$ corresponds to
$\left[\left(\begin{smallmatrix}j\\0\end{smallmatrix}\right)\right]\in \Lambda$.

We have
$$\text{NN}(j) = \{(j\pm 1)\%12,(j\pm 3)\%12\},\qquad \text{NNN}(j) = \{(j\pm 2)\%12, (j\pm 4)\%12\}.$$

For implementation, we represent $\uparrow$ with $0$, $\downarrow$ with $1$, and order the creation operators as follows:
$$
  c_{(\left|\Lambda\right|-1)\downarrow}^\dagger \cdots c_{0\downarrow}^\dagger
  c_{(\left|\Lambda\right|-1)  \uparrow}^\dagger \cdots c_{0  \uparrow}^\dagger
  \left|0\right>
$$

In [7]:
n_configs_per_spin = scipy.special.comb(ham_family.nsites, ham_family.nsites // 2, exact=True)
print(f"Number of possible bitstrings in Hubbard{ham_family.nsites} is {n_configs_per_spin}**2 = {n_configs_per_spin**2}.")

Number of possible bitstrings in Hubbard12 is 924**2 = 853776.


## Data generation

### Lanczos

* Requires: nothing
* Generates: ground state vectors and probabilities in lanczos dir (but takes weeks to do so, use HPC instead)

In [9]:
def compute_lanczos_f(ncv, maxiter):
    def compute_lanczos(params_vec):
        return classifim_gen.gs_utils.compute_lanczos(
            params_vec,
            ham_family=ham_family,
            k=4,
            ncv=ncv,
            maxiter=maxiter,
            beta=1.0e7,
            payload=None,
            verbose=False
        )
    return compute_lanczos
resolution = 64
lambdas = np.array([(lambdai0 / resolution, lambdai1 / resolution) for lambdai0 in range(resolution) for lambdai1 in range(resolution)])
param_vecs = np.array(param_conversions.lambdas_to_params(*lambdas.T)).T
DATA_LANCZOS_DIR = classifim.utils.maybe_create_subdir(DATA_DIR, "lanczos")
lanczos_cache = classifim.bench.GroundStateCache(
    # ncv and maxiter need to be increased for some points.
    compute_ground_state=compute_lanczos_f(40, 40),
    param_keys=ham_family.PARAM_NAMES,
    ham_name=SM_NAME.lower(),
    save_path=DATA_LANCZOS_DIR,
    load_meta=True,
    filename_source=classifim_gen.gs_cache.FS_FILESYSTEM
)

In [ ]:
# This should compute "ground state" for all values of parameters in theory.
# Takes weeks in practice.
LOCK_FILE_PATH = os.path.join(DATA_LANCZOS_DIR, f"{SM_NAME.lower()}_bench.lock")
num_success = 0
num_error = 0
num_skipped = 0
done = False
with classifim_gen.gs_utils.ProcessLockFile(LOCK_FILE_PATH) as process_lock:
    print(f"Locked using '{LOCK_FILE_PATH}'. Remove to stop softly.")
    try:
        for ncv in (40, 60, 90, 130, 200):
            if done or num_success + num_skipped >= param_vecs.shape[0]:
                break
            num_success = 0
            num_error = 0
            num_skipped = 0
            # Use maxiter = ncv. This can be optimized further.
            lanczos_cache.compute_ground_state = compute_lanczos_f(ncv, maxiter=ncv)
            progress_printer = classifim.bench.ProgressPrinter()
            progress_printer.pprintln(f"Start time: {progress_printer.get_now_str()}")
            for param_vec in param_vecs:
                if not process_lock.exists():
                    progress_printer.pprintln(f"\rLock file '{LOCK_FILE_PATH}' removed; exiting.")
                    done = True
                    break
                try:
                    res = lanczos_cache.get_ground_state(tuple(param_vec), load=False)
                except classifim_gen.gs_cache.GroundStateComputationError:
                    res = 'error'
                if res is None:
                    num_skipped += 1
                    progress_printer.inc_i(char='.')
                elif res == 'error':
                    num_error += 1
                    progress_printer.inc_i(char='E')
                else:
                    num_success += 1
                    progress_printer.inc_i(char='S')
            progress_printer.pprintln(f"\r{ncv=} done: {num_success=} {num_error=} {num_skipped=}")
    finally:
        lanczos_cache.save_meta()
        progress_printer.pprintln(f"\rEnd time: {progress_printer.get_now_str()}")

### Generate datasets

* Requires: ground state probabilities in "lanczos" directory.
* Generates: classifim_datasets

In [ ]:
%%time
BITCHIFC_DATASET_DIR = classifim.utils.maybe_create_subdir(DATA_DIR, "classifim_datasets")
lanczos_cache = get_lanczos_cache()
resolution = 64
lambdas = np.array([
    (lambda0, lambda1)
    for lambda0 in np.arange(resolution) / resolution
    for lambda1 in np.arange(resolution) / resolution])
params_vecs = np.array(param_conversions.lambdas_to_params(*lambdas.T)).T
print(f"{params_vecs.shape=}")
datasets = classifim_gen.gs_utils.generate_datasets(
    gs_cache=lanczos_cache,
    lambdas=lambdas,
    params_vecs=params_vecs,
    vi_to_z=ham_family.vi_to_z,
    seeds=range(42, 52))

for dataset in datasets:
    seed = dataset["seed"]
    dataset_file_name =  os.path.join(BITCHIFC_DATASET_DIR, f"dataset_{seed}.npz")
    np.savez_compressed(dataset_file_name, **dataset)
    print(f"Dataset is saved to '{dataset_file_name}'")

### Compute ground truth FIM

* Requires: ground state probabilities in "lanczos" directory.
* Generates: ground truth FIM (`fim/gs_fim.npz`)

In [ ]:
%%time
FIM_DIR = classifim.utils.maybe_create_subdir(DATA_DIR, "fim")
lanczos_cache = get_lanczos_cache()

gs_fim = classifim.bench.fidelity.compute_2d_fim(
    param_conversions.lambdas_to_params, lanczos_cache,
    resolution=64, verbose=True)

gs_fim_npz = {}
for key, value in gs_fim.items():
    np_value = value.to_numpy()
    if np_value.dtype == object:
        np_value = np_value.astype(bytes)
    gs_fim_npz[key] = np_value

gs_fim_filename = os.path.join(FIM_DIR, "gs_fim.npz")
np.savez_compressed(
    gs_fim_filename,
    **gs_fim_npz)
print(f"gs_fim saved to '{gs_fim_filename}'")

### Convert to parquet

In [29]:
importlib.reload(classifim_gen.io)
importlib.reload(classifim_gen.hubbard_hamiltonian)
data_seeds = classifim_gen.io.save_datasets_for_hf(
    convert_f=classifim_gen.hubbard_hamiltonian.convert_dataset_to_hf,
    input_pattern=os.path.join(DATA_DIR, "classifim_datasets", "dataset_{seed}.npz"),
    output_dir=HF_DATA_DIR,
    overwrite=True)
print("Done.")

Done.


In [24]:
with np.load(os.path.join(DATA_DIR, "fim", "gs_fim.npz")) as f:
    fim_npz = dict(f)
gt_fim_filename = os.path.join(HF_DATA_DIR, "gt_fim.parquet")
pq.write_table(pa.Table.from_pydict(fim_npz), gt_fim_filename)

In [27]:
import importlib
importlib.reload(classifim_gen.io)
print(classifim_gen.io.gen_config_yml(
    sm_name=os.path.basename(HF_DATA_DIR),
    seeds=sorted(data_seeds),
    fim_seeds=None))

- config_name: hubbard_12.gt_fim
  data_files:
  - split: test
    path: hubbard_12/gt_fim.parquet
- config_name: hubbard_12.seed42
  data_files:
  - split: train
    path: hubbard_12/seed_42/d_train.parquet
  - split: test
    path: hubbard_12/seed_42/d_test.parquet
- config_name: hubbard_12.seed43
  data_files:
  - split: train
    path: hubbard_12/seed_43/d_train.parquet
  - split: test
    path: hubbard_12/seed_43/d_test.parquet
- config_name: hubbard_12.seed44
  data_files:
  - split: train
    path: hubbard_12/seed_44/d_train.parquet
  - split: test
    path: hubbard_12/seed_44/d_test.parquet
- config_name: hubbard_12.seed45
  data_files:
  - split: train
    path: hubbard_12/seed_45/d_train.parquet
  - split: test
    path: hubbard_12/seed_45/d_test.parquet
- config_name: hubbard_12.seed46
  data_files:
  - split: train
    path: hubbard_12/seed_46/d_train.parquet
  - split: test
    path: hubbard_12/seed_46/d_test.parquet
- config_name: hubbard_12.seed47
  data_files:
  - split

## Best possible cross entropy error

* Requires:
    - Ground state probabilities in 'lanczos' dir
    - Test dumps (produced by `twelve_sites_classifim.ipynb`) at `classifim_datasets/test_dump_{seed}.npz`.
* Prints out best possible cross entropy error (for an algorithm which knows ground state probabilities).

#### Compute and save

In [16]:
# %%time
DATASETS_DIR = os.path.join(DATA_DIR, "classifim_datasets")
assert os.path.isdir(DATASETS_DIR)

def zs_to_vi_f(zs):
    z_up = zs & ((1 << 12) - 1)
    z_down = zs >> 12
    vi = ham_family.z_to_vi(z_down, z_up)
    return vi

best_xes = {}
for seed in range(42, 52):
    res = classifim.bench.fil24_hamiltonian.compute_best_possible_xe(
        dump_npz=np.load(os.path.join(DATASETS_DIR, f"test_dump_{seed}.npz")),
        zs_to_vi_f=zs_to_vi_f,
        lambdas_to_params_f=param_conversions.lambdas_to_params,
        probs_cache=get_lanczos_cache(),
        resolution=64)
    best_xes[seed] = res

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4096/4096 [01:59<00:00, 34.15it/s]


In [17]:
METRICS_DIR = os.path.join(DATA_DIR, "metrics")
best_xes_filename = os.path.join(METRICS_DIR, "best_xes.npz")
np.savez_compressed(
    best_xes_filename,
    seeds=np.array(list(best_xes.keys())),
    best_xes=np.array(list(best_xes.values())))

#### Load and print

In [19]:
METRICS_DIR = os.path.join(DATA_DIR, "metrics")
best_xes_filename = os.path.join(METRICS_DIR, "best_xes.npz")
with np.load(best_xes_filename) as f:
    best_xes = dict(f)
summary = classifim.bench.metric.normal_summary(best_xes["best_xes"])
print(f"Best possible XE: {summary}")

Best possible XE: 0.2380 \pm 0.0009


# Scratch

In [8]:
with np.load(os.path.join(DATA_DIR, "classifim_datasets/dataset_42.npz")) as f:
    npz_ds = dict(f)

In [9]:
npz_ds

{'lambdas': array([[0.      , 0.      ],
        [0.      , 0.      ],
        [0.      , 0.      ],
        ...,
        [0.984375, 0.984375],
        [0.984375, 0.984375],
        [0.984375, 0.984375]]),
 'zs': array([13845195,  5737095,  8030295, ..., 10647372, 11626380,   653989],
       dtype=uint32),
 'seed': array(42)}

In [10]:
for k, v in npz_ds.items():
    v_info = v.item() if v.size == 1 else v.shape
    print(f"{k}: {v.dtype} {v_info}")

lambdas: float64 (573440, 2)
zs: uint32 (573440,)
seed: int64 42
